In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf, SQLContext

In [3]:
sc=SparkContext()

In [4]:
spark=SQLContext(sc)

In [5]:
project=spark.read.csv("project.csv", header= True)
country=spark.read.csv("country.csv", header= True)
proceeds=spark.read.csv("proceeds.csv", header= True)
category=spark.read.csv("category.csv", header= True)

In [6]:
project.show(2)
country.show(2)
proceeds.show(2)
category.show(2)

+----------+--------------------+-------------------+----------+------+----------+-----------+
|project_id|           proj_name|           launched|  deadline| state|country_id|category_id|
+----------+--------------------+-------------------+----------+------+----------+-----------+
|1000002330|The Songs of Adel...|2015-08-11 12:12:28|2015-10-09|failed|        10|        117|
|1000003930|Greeting From Ear...|2017-09-02 04:43:57|2017-11-01|failed|        22|        102|
+----------+--------------------+-------------------+----------+------+----------+-----------+
only showing top 2 rows

+------------+--------+----------+
|country_code|currency|country_id|
+------------+--------+----------+
|          AT|     EUR|         1|
|          AU|     AUD|         2|
+------------+--------+----------+
only showing top 2 rows

+-------+-------+-------+-----------+--------+----------+-----------+
|   goal|pledged|backers|usd_pledged|usd_goal|project_id|proceeds_id|
+-------+-------+-------+-----

In [7]:
project.count()

390736

In [8]:
project.columns

['project_id',
 'proj_name',
 'launched',
 'deadline',
 'state',
 'country_id',
 'category_id']

->>>>>>>>    Importing sql functions   <<<<<<<<<<

In [9]:
from pyspark.sql.functions import col, desc, asc, count, min,max, year, last

# 1. How many successful projects with zero backers?

In [10]:
self_join= project.join(proceeds, project.project_id==proceeds.project_id, "inner").select(project.state, proceeds.backers)

In [11]:
self_join.show(10)

+----------+-------+
|     state|backers|
+----------+-------+
|    failed|      0|
|    failed|     15|
|    failed|      3|
|    failed|      1|
|  canceled|     14|
|successful|    224|
|successful|     16|
|    failed|     40|
|  canceled|     58|
|  canceled|     43|
+----------+-------+
only showing top 10 rows



In [12]:
successful_backer= self_join.filter((self_join.state=="successful") & (self_join.backers=="0"))

In [13]:
successful_backer.count()

0

# 2. Which country has the least amount of projects?

In [15]:
data = project.join(country, project.country_id==country.country_id, "inner").select(project.proj_name, country.country_code)

In [16]:
data.show()

+--------------------+------------+
|           proj_name|country_code|
+--------------------+------------+
|The Songs of Adel...|          GB|
|Greeting From Ear...|          US|
|      Where is Hank?|          US|
|ToshiCapital Reko...|          US|
|Community Film Pr...|          US|
|Monarch Espresso Bar|          US|
|Support Solar Roa...|          US|
|Chaser Strips. Ou...|          US|
|SPIN - Premium Re...|          US|
|STUDIO IN THE SKY...|          US|
| Of Jesus and Madmen|          CA|
|    Lisa Lim New CD!|          US|
|  The Cottage Market|          US|
|G-Spot Place for ...|          US|
|Tombstone: Old We...|          GB|
|      Survival Rings|          US|
|           The Beard|          US|
|Notes From London...|          US|
|Mike Corey's Dark...|          US|
|            Boco Tea|          US|
+--------------------+------------+
only showing top 20 rows



In [17]:
data1=data.groupBy("country_code").count().sort(col("count").asc())

In [18]:
data1.show()

+------------+-----+
|country_code|count|
+------------+-----+
|          JP|   40|
|          LU|   65|
|          SG|  570|
|          HK|  622|
|          AT|  624|
|          BE|  643|
|          NO|  761|
|          CH|  812|
|          IE|  886|
|          DK| 1201|
|          NZ| 1533|
|          SE| 1864|
|          MX| 1888|
|          ES| 2389|
|          NL| 3011|
|          IT| 3062|
|          FR| 3113|
|          DE| 4414|
|          AU| 8384|
|          CA|15633|
+------------+-----+
only showing top 20 rows



In [19]:
data1.first()

Row(country_code='JP', count=40)

# 3. List the top 10 projects that have the highest goal and list the ones that have reached their goals.

In [20]:
df= project.join(proceeds, project.project_id==proceeds.project_id).select(project.proj_name, proceeds.goal, proceeds.pledged)

In [21]:
df.show()

+--------------------+---------+--------+
|           proj_name|     goal| pledged|
+--------------------+---------+--------+
|The Songs of Adel...|  1000.00|    0.00|
|Greeting From Ear...| 30000.00| 2421.00|
|      Where is Hank?| 45000.00|  220.00|
|ToshiCapital Reko...|  5000.00|    1.00|
|Community Film Pr...| 19500.00| 1283.00|
|Monarch Espresso Bar| 50000.00|52375.00|
|Support Solar Roa...|  1000.00| 1205.00|
|Chaser Strips. Ou...| 25000.00|  453.00|
|SPIN - Premium Re...|125000.00| 8233.00|
|STUDIO IN THE SKY...| 65000.00| 6240.57|
| Of Jesus and Madmen|  2500.00|    0.00|
|    Lisa Lim New CD!| 12500.00|12700.00|
|  The Cottage Market|  5000.00|    0.00|
|G-Spot Place for ...|200000.00|    0.00|
|Tombstone: Old We...|  5000.00|94175.00|
|      Survival Rings|  2500.00|  664.00|
|           The Beard|  1500.00|  395.00|
|Notes From London...|  3000.00|  789.00|
|Mike Corey's Dark...|   250.00|  250.00|
|            Boco Tea|  5000.00| 1781.00|
+--------------------+---------+--

In [22]:
df1=df.orderBy(col("goal").desc())

In [23]:
df1.show(10, truncate =False)

+-----------------------------------------------------------+---------+--------+
|proj_name                                                  |goal     |pledged |
+-----------------------------------------------------------+---------+--------+
|'Laborer App' (Canceled)                                   |999999.00|11.00   |
|GTA5 Devin Westin's house in real life (Canceled)          |999999.00|0.00    |
|Anup R                                                     |999999.00|22.00   |
|TRANSFORMATION TECHNOLOGY                                  |99999.99 |6.00    |
|999Bottles                                                 |99999.00 |57362.81|
|Babyltop Soccer                                            |99999.00 |100.00  |
|White House Toy Play Set                                   |99999.00 |175.00  |
|World Most Exclusive Polo! Limited Handmade, Ping Pong Bomb|99999.00 |200.00  |
|Clean Grip - Protect your grip, Travel in Style.           |99999.00 |86.00   |
|IndeChat                   

In [24]:
proceeds.show()

+---------+-------+-------+-----------+---------+----------+-----------+
|     goal|pledged|backers|usd_pledged| usd_goal|project_id|proceeds_id|
+---------+-------+-------+-----------+---------+----------+-----------+
|  1000.00|   0.00|      0|       0.00|  1533.95|1000002330|          1|
|  2500.00|   0.00|      0|       0.00|  2406.39| 100004721|          2|
|  5000.00|   0.00|      0|       0.00|  5000.00|1000055792|          3|
|200000.00|   0.00|      0|       0.00|200000.00|1000056157|          4|
| 12000.00|   0.00|      0|       0.00| 17489.65|1000103948|          5|
|  2500.00|   0.00|      0|       0.00|  2500.00|1000182258|          6|
| 13500.00|   0.00|      0|       0.00| 13500.00|1000226576|          7|
| 50000.00|   0.00|      0|       0.00| 39739.31|1000245024|          8|
|  2000.00|   0.00|      0|       0.00|  2579.35|1000256230|          9|
| 25000.00|   0.00|      0|       0.00| 25000.00|1000282287|         10|
| 68000.00|   0.00|      0|       0.00| 68000.00|10

In [25]:
reached_goals=df.select("proj_name","goal","pledged").filter(df.pledged>=df.goal)

In [26]:
reached_goals.show(truncate=False)

+------------------------------------------------------------+---------+--------+
|proj_name                                                   |goal     |pledged |
+------------------------------------------------------------+---------+--------+
|Monarch Espresso Bar                                        |50000.00 |52375.00|
|Support Solar Roasted Coffee & Green Energy!  SolarCoffee.co|1000.00  |1205.00 |
|Chaser Strips. Our Strips make Shots their B*tch!           |25000.00 |453.00  |
|SPIN - Premium Retractable In-Ear Headphones with Mic       |125000.00|8233.00 |
|Lisa Lim New CD!                                            |12500.00 |12700.00|
|Tombstone: Old West tabletop game and miniatures in 32mm.   |5000.00  |94175.00|
|Survival Rings                                              |2500.00  |664.00  |
|The Beard                                                   |1500.00  |395.00  |
|Notes From London: Above & Below                            |3000.00  |789.00  |
|Mike Corey's Da

In [27]:
reached_goals.count()

217220

# 4. Which category has the most number of projects?

In [28]:
cat=project.join(category,project.category_id == category.category_id,"inner").select("proj_name","category")

In [29]:
cat.show()

+--------------------+--------------+
|           proj_name|      category|
+--------------------+--------------+
|The Songs of Adel...|        Poetry|
|Greeting From Ear...|Narrative Film|
|      Where is Hank?|Narrative Film|
|ToshiCapital Reko...|         Music|
|Community Film Pr...|  Film & Video|
|Monarch Espresso Bar|   Restaurants|
|Support Solar Roa...|          Food|
|Chaser Strips. Ou...|        Drinks|
|SPIN - Premium Re...|Product Design|
|STUDIO IN THE SKY...|   Documentary|
| Of Jesus and Madmen|    Nonfiction|
|    Lisa Lim New CD!|    Indie Rock|
|  The Cottage Market|        Crafts|
|G-Spot Place for ...|         Games|
|Tombstone: Old We...|Tabletop Games|
|      Survival Rings|        Design|
|           The Beard|   Comic Books|
|Notes From London...|     Art Books|
|Mike Corey's Dark...|         Music|
|            Boco Tea|          Food|
+--------------------+--------------+
only showing top 20 rows



In [30]:
cat1=cat.groupby("category").count().sort(col("count").desc())

In [31]:
cat1.show()

+----------------+-----+
|        category|count|
+----------------+-----+
|  Product Design|22274|
|     Documentary|16071|
|  Tabletop Games|14166|
|           Music|13275|
|          Shorts|12280|
|     Video Games|11820|
|            Food|11475|
|             Web|10292|
|          Comedy| 9240|
|    Film & Video| 9186|
|         Fiction| 9099|
|         Fashion| 8543|
|      Nonfiction| 8265|
|             Art| 8233|
|         Apparel| 7156|
|         Theater| 6977|
|      Technology| 6921|
|            Rock| 6713|
|Children's Books| 6699|
|            Apps| 6338|
+----------------+-----+
only showing top 20 rows



In [32]:
cat1.first()

Row(category='Product Design', count=22274)

# 5. Which category is the most successful?


In [33]:
success=project.join(category,project.category_id == category.category_id,"inner").select("state","category").filter(project.state=="successful")

In [34]:
success.show()

+----------+--------------+
|     state|      category|
+----------+--------------+
|successful|   Restaurants|
|successful|          Food|
|successful|    Indie Rock|
|successful|Tabletop Games|
|successful|         Music|
|successful|       Fashion|
|successful|        Comics|
|successful|         Music|
|successful|     Webseries|
|successful|        Comics|
|successful|     Animation|
|successful|Product Design|
|successful|Product Design|
|successful|         Music|
|successful|    Indie Rock|
|successful|   Documentary|
|successful|    Indie Rock|
|successful|         Music|
|successful|   Photography|
|successful|           Pop|
+----------+--------------+
only showing top 20 rows



In [35]:
success_cat=success.groupBy("category").count().sort(col("count").desc())

In [36]:
success_cat.show()

+----------------+-----+
|        category|count|
+----------------+-----+
|  Product Design| 7953|
|  Tabletop Games| 7863|
|          Shorts| 6630|
|           Music| 6303|
|     Documentary| 5905|
|         Theater| 4361|
|          Comedy| 3624|
|      Indie Rock| 3585|
|            Rock| 3476|
|             Art| 3376|
|            Food| 3294|
|    Film & Video| 3048|
|  Country & Folk| 2790|
|          Comics| 2495|
|     Video Games| 2386|
|Children's Books| 2327|
|      Nonfiction| 2216|
|         Fiction| 2213|
|         Fashion| 2186|
|  Narrative Film| 2004|
+----------------+-----+
only showing top 20 rows



In [37]:
success_cat.first()

Row(category='Product Design', count=7953)

# 6. What type of projects are prevalent in each country?

In [38]:
dataa = project.join(country, project.country_id==country.country_id, "inner").join(category, project.category_id==category.category_id).select(project.proj_name, country.country_code, category.category, category.main_category)

In [39]:
dataa.show()

+--------------------+------------+--------------+-------------+
|           proj_name|country_code|      category|main_category|
+--------------------+------------+--------------+-------------+
|The Songs of Adel...|          GB|        Poetry|   Publishing|
|Greeting From Ear...|          US|Narrative Film| Film & Video|
|      Where is Hank?|          US|Narrative Film| Film & Video|
|ToshiCapital Reko...|          US|         Music|        Music|
|Community Film Pr...|          US|  Film & Video| Film & Video|
|Monarch Espresso Bar|          US|   Restaurants|         Food|
|Support Solar Roa...|          US|          Food|         Food|
|Chaser Strips. Ou...|          US|        Drinks|         Food|
|SPIN - Premium Re...|          US|Product Design|       Design|
|STUDIO IN THE SKY...|          US|   Documentary| Film & Video|
| Of Jesus and Madmen|          CA|    Nonfiction|   Publishing|
|    Lisa Lim New CD!|          US|    Indie Rock|        Music|
|  The Cottage Market|   

In [40]:
 dataa1 = dataa.groupby('country_code','category').count().sort(col("count").desc())
dataa1.show()

+------------+----------------+-----+
|country_code|        category|count|
+------------+----------------+-----+
|          US|  Product Design|16193|
|          US|     Documentary|13610|
|          US|           Music|11532|
|          US|  Tabletop Games|10681|
|          US|          Shorts|10273|
|          US|            Food| 9648|
|          US|    Film & Video| 7884|
|          US|     Video Games| 7760|
|          US|         Fiction| 7611|
|          US|         Fashion| 6912|
|          US|      Nonfiction| 6860|
|          US|          Comedy| 6764|
|          US|             Art| 6478|
|          US|             Web| 6280|
|          US|         Theater| 6122|
|          US|            Rock| 6038|
|          US|Children's Books| 5558|
|          US|      Indie Rock| 5261|
|          US|         Apparel| 5205|
|          US|       Webseries| 4990|
+------------+----------------+-----+
only showing top 20 rows



In [41]:
dataa1.drop_duplicates(["country_code"]).show(22)

+------------+--------------+-----+
|country_code|      category|count|
+------------+--------------+-----+
|          NL|Product Design|  264|
|          MX|   Video Games|   85|
|          AT|Product Design|   58|
|          HK|Product Design|  217|
|          AU|Product Design|  697|
|          CA|Product Design| 1117|
|          GB|Product Design| 1839|
|          DE|Product Design|  425|
|          ES|   Video Games|  200|
|          US|Product Design|16193|
|          FR|           Web|  260|
|          CH|Product Design|  107|
|          SG|Product Design|  121|
|          IT|Product Design|  279|
|          SE|Product Design|  134|
|          JP|Product Design|    6|
|          NZ|Product Design|  105|
|          IE|           Web|   62|
|          BE|   Video Games|   48|
|          NO|           Web|   56|
|          LU|           Web|    6|
|          DK|Product Design|  129|
+------------+--------------+-----+



# 7. Which year has the most number of failed projects?

In [42]:
year=project.select((year('deadline').alias('year')),"state").filter(project.state=="failed")

In [43]:
year.show()

+----+------+
|year| state|
+----+------+
|2015|failed|
|2017|failed|
|2013|failed|
|2012|failed|
|2016|failed|
|2013|failed|
|2014|failed|
|2016|failed|
|2015|failed|
|2014|failed|
|2015|failed|
|2012|failed|
|2017|failed|
|2014|failed|
|2015|failed|
|2016|failed|
|2015|failed|
|2013|failed|
|2013|failed|
|2016|failed|
+----+------+
only showing top 20 rows



In [44]:
year_with_max_fail=year.groupBy("year").count().sort(col("count").desc())
year_with_max_fail.show()

+----+-----+
|year|count|
+----+-----+
|2015|47978|
|2014|38985|
|2016|33074|
|2017|28935|
|2013|21214|
|2012|20646|
|2011|11116|
|2010| 4245|
|2009|  407|
|2018|  144|
+----+-----+



In [45]:
year_with_max_fail.first()

Row(year=2015, count=47978)